In [ ]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

Importar la base de datos de imdb (dataset)

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file("aclImdb.tar.gz", url,
                                  untar = True, cache_dir= '.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84125825/84125825 [==============================] - 2s 0us/step


['README', 'imdbEr.txt', 'test', 'train', 'imdb.vocab']

Eliminar las carpetas con informacion que no vamos a usar

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Crear dataset

In [ ]:
batch_size = 1024
seed = 123

train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset = 'training', seed=seed)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset = 'validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Capa Text Embedding

In [ ]:
embedding_layer = tf.keras.layers.Embedding(1000,5)
result = embedding_layer(tf.constant([1, 2, 3]))
print(result.numpy())

[[-0.04444961 -0.03410434 -0.03669883 -0.03005648  0.02759335]
 [-0.03323843 -0.04453628 -0.03266414 -0.04871795 -0.02273521]
 [ 0.04046943  0.04553935  0.01093223 -0.03305067  0.00463427]]


In [ ]:
# 3 palabras diferentes, 5 dimensiones
print(result.shape)

(3, 5)


Preprocesamiento de Texto

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

vocab_size = 10000
sequence_length = 100

# Creamos el vector de texto
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

# Adaptamos el vectorizador al dataset de entrenamiento
text_ds = train_ds.map(lambda x, y: x)  # Asegúrate de tener train_ds definido
vectorize_layer.adapt(text_ds)


Crear modelo de clasificacion

In [ ]:
embedding_dim = 16
model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, name='embedding'),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 15
)

Epoch 1/15
20/20 [==============================] - 3s 116ms/step - loss: 0.6922 - accuracy: 0.5028 - val_loss: 0.6907 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 83ms/step - loss: 0.6878 - accuracy: 0.5028 - val_loss: 0.6852 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 85ms/step - loss: 0.6802 - accuracy: 0.5028 - val_loss: 0.6761 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 109ms/step - loss: 0.6683 - accuracy: 0.5028 - val_loss: 0.6625 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 86ms/step - loss: 0.6511 - accuracy: 0.5028 - val_loss: 0.6440 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 2s 85ms/step - loss: 0.6287 - accuracy: 0.5031 - val_loss: 0.6212 - val_accuracy: 0.4898
Epoch 7/15
20/20 [==============================] - 2s 88ms/step - loss: 0.6016 - accuracy: 0.5457 - val_loss: 0.5951 - val_accuracy: 0.5700
Epoch 8/15


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160289 (626.13 KB)
Trainable params: 16028

Descarga de datos

In [ ]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + '\n')

out_v.close()
out_m.close()